# (연습) 모집단과 표본

**기본 설정**

Numpy와 Pandas 라이브러리를 각각 np와 pd로 불러온다.

In [1]:
import numpy as np
import pandas as pd

데이터프레임의 [chained indexing을 금지시키기 위한 설정](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy)을 지정한다.
Pandas 3.0 버전부터는 기본 옵션으로 지정된다.

In [2]:
pd.options.mode.copy_on_write = True

주피터 노트북에서 부동소수점의 출력을 소수점 이하 6자리로 제한한다.
아래 코드는 주피터 노트북에서만 사용하며 일반적인 파이썬 코드가 아니다.

In [3]:
%precision 6

'%.6f'

아래 코드는 데이터프레임 내에서 부동소수점의 출력을 소수점 이하 6자리로 제한한다.

In [4]:
pd.set_option('display.precision', 6)

데이터 시각화를 위해 `matplotlib.pyplot`를 `plt`라는 별칭으로 불러온다.

In [5]:
import matplotlib.pyplot as plt

**데이터 저장소 디렉토리**

코드에 사용되는 [데이터 저장소의 기본 디렉토리](https://github.com/codingalzi/DataSci/tree/master/data)를 지정한다.

In [6]:
data_url = 'https://raw.githubusercontent.com/codingalzi/DataSci/refs/heads/master/data/'

## 데이터셋 불러오기

**타이타닉 데이터셋**

In [7]:
titanic = pd.read_csv(data_url+"titanic.csv")

12개의 특성을 갖는다.

| 특성 | 의미 |
| :--- | :--- |
| PassengerId  | 승객 번호 |
| Survived | 생존 여부. 0 또는 1. 1일 때 생존 |
| Pclass | 승객 클래스 |
| Name | 승객 이름 |
| Sex | 승객의 성 |
| Age | 승객 나이 |
| SibSp | 타이타닉에 함께 승선한 형제자매와 배우자의 수 |
| Parch | 타이타닉에 함께 승선한 자녀와 부모의 수 |
| Ticket | 티켓 번호 |
| Fare | 티켓 요금(영구 파운드화) |
| Cabin | 객실 번호 |
| Embarked | 승객이 타이타닉호에 승선한 항구 |

In [147]:
titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


`PassengerId` 특성을 인덱스로 지정한다.

In [8]:
titanic = titanic.set_index("PassengerId")

**목표**

승객의 생존에 영향을 많이 끼친 특성을 파악한다. 

## 결측치 처리

**결측치 확인**

데이터셋의 크기인 891보다 적은 수의 `non-null` 값을 갖는 특성에 결측치가 존재한다.
데이터프레임의 `info()` 메서드로 확인하면 `Age`, `Cabin`, `Embarked` 특성에 
아래 표에 언급된 만큼의 결측치가 포함된다.

| 특성 | 결측치 수 |
| :--- | :---: |
| `Age` | 177 |
| `Cabin` | 687 |
| `Embarked` | 2 |

In [9]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


- `Cabin` 특성 결측치: 
    - 약 77%가 결측치다.
    - 이처럼 결측치의 비율이 너무 큰 경우엔 해당 특성을 무사하는 게 좋다.
- `Age` 특성 결측치
    - 보통 평균값 또는 중앙값으로 대체한다.
    - 경우에 따라 성별 또는 나이대별 나이의 평균값, 중앙값 등으로 대체하기도 한다.
    - 어떤 방식이 데이터분석의 목적에 보다 적합한가는 미리 알 수 없다.
- `Embarked` 특성 결측치
    - 범주형 특성의 결측치는 일반적으로 최빈값으로 대체한다.
    - 게다가 결측치가 2개밖에 없기에 최빈값으로 대체해도 데이터분석에 별 영향을 주지 않을 것이다.

**`Embarked` 특성 결측치 처리**

`Embarked` 특성의 결측치를 모두 해당 특성의 최빈값으로 대체한다.

In [14]:
embarked_mode =titanic_med['Embarked'].mode()
embarked_mode

0    S
Name: Embarked, dtype: object

In [15]:
titanic['Embarked'] = titanic['Embarked'].fillna(embarked_mode)

In [20]:
mask = titanic['Embarked'].isna()

In [21]:
mask.sum()

2

In [22]:
titanic[mask]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN
830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN


In [19]:
titanic['Embarked'].fillna(embarked_mode).info()

<class 'pandas.core.series.Series'>
Index: 891 entries, 1 to 891
Series name: Embarked
Non-Null Count  Dtype 
--------------  ----- 
889 non-null    object
dtypes: object(1)
memory usage: 13.9+ KB


`Embarked` 특성의 결측치는 이제 더 이상 없다.

In [16]:
titanic['Embarked'].info()

<class 'pandas.core.series.Series'>
Index: 891 entries, 1 to 891
Series name: Embarked
Non-Null Count  Dtype 
--------------  ----- 
889 non-null    object
dtypes: object(1)
memory usage: 13.9+ KB


In [17]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


**`Age` 특성 결측치 처리: 특성 중앙값 활용**

데이터셋을 복제해서 활용한다.

In [11]:
titanic_med = titanic.copy()

`Age` 특성의 결측치를 모두 해당 특성의 중앙값으로 대체한다.

In [12]:
age_median =titanic_med['Age'].median()
age_median

28.000000

In [13]:
titanic_med['Age'] = titanic_med['Age'].fillna(age_median)

In [109]:
titanic_med.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [110]:
titanic_med['Embarked'].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

In [111]:
titanic_med['Embarked'].mode()

0    S
Name: Embarked, dtype: object

In [112]:
titanic_med['Embarked'] = titanic_med['Embarked'].fillna(titanic_med['Embarked'].mode())

In [113]:
titanic_med.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [114]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


**성에 따라 다른 중앙값 계산**

In [159]:
titanic

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [150]:
titanic_sex = titanic.copy()

In [163]:
titanic.iloc[878]

Survived                     0
Pclass                       3
Name        Laleff, Mr. Kristo
Sex                       male
Age                        NaN
SibSp                        0
Parch                        0
Ticket                  349217
Fare                    7.8958
Cabin                      NaN
Embarked                     S
Name: 879, dtype: object

In [160]:
titanic_sex.groupby('Sex')['Age'].apply(lambda y:y.fillna(y.median()))[-10:]

Sex   PassengerId
male  874            47.0
      877            20.0
      878            19.0
      879            29.0
      882            33.0
      884            28.0
      885            25.0
      887            27.0
      890            26.0
      891            32.0
Name: Age, dtype: float64

In [124]:
f_mask = titanic["Sex"]=="female"
m_mask = titanic["Sex"]=="male"

In [125]:
titanic[f_mask]["Age"].median()

27.000000

In [126]:
titanic[m_mask]["Age"].median()

29.000000

In [120]:
titanic.groupby('Sex')['Age'].median()

Sex
female    27.0
male      29.0
Name: Age, dtype: float64

In [140]:
titanic.loc[f_mask, 'Age'] = titanic[f_mask]['Age'].fillna(27.0)

In [145]:
titanic.loc[m_mask, 'Age'] = titanic[f_mask]['Age'].fillna(29.0)

In [143]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       891 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [144]:
titanic

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,29.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,29.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,29.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,27.0,1,2,W./C. 6607,23.4500,NaN,S


The **Name** and **Ticket** attributes may have some value, but they will be a bit tricky to convert into useful numbers that a model can consume. So for now, we will ignore them.

Let's take a look at the numerical attributes:

In [62]:
titanic.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


* Yikes, only 38% **Survived**! 😭 That's close enough to 40%, so accuracy will be a reasonable metric to evaluate our model.
* The mean **Fare** was £32.20, which does not seem so expensive (but it was probably a lot of money back then).
* The mean **Age** was less than 30 years old.

Let's check that the target is indeed 0 or 1:

In [63]:
titanic["Survived"].value_counts()

Survived
0    549
1    342
Name: count, dtype: int64

Now let's take a quick look at all the categorical attributes:

In [64]:
titanic["Pclass"].value_counts()

Pclass
3    491
1    216
2    184
Name: count, dtype: int64

In [65]:
titanic["Sex"].value_counts()

Sex
male      577
female    314
Name: count, dtype: int64

In [66]:
titanic["Embarked"].value_counts()

Embarked
S    644
C    168
Q     77
Name: count, dtype: int64

The Embarked attribute tells us where the passenger embarked: C=Cherbourg, Q=Queenstown, S=Southampton.

* Try to convert numerical attributes to categorical attributes: for example, different age groups had very different survival rates (see below), so it may help to create an age bucket category and use it instead of the age. Similarly, it may be useful to have a special category for people traveling alone since only 30% of them survived (see below).
* Replace **SibSp** and **Parch** with their sum.
* Try to identify parts of names that correlate well with the **Survived** attribute.
* Use the **Cabin** column, for example take its first letter and treat it as a categorical attribute.

In [67]:
titanic["AgeBucket"] = titanic["Age"] // 15 * 15

In [68]:
titanic

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,AgeBucket
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,15.0
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,30.0
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,15.0
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,30.0
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,15.0
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,15.0
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,NaN


In [69]:
titanic[["AgeBucket", "Survived"]].groupby(['AgeBucket']).mean()

,Survived
AgeBucket,
0.0,0.576923
15.0,0.362745
30.0,0.423256
45.0,0.404494
60.0,0.240000
75.0,1.000000


In [70]:
titanic[["Sex", "Survived"]].groupby(['Sex']).mean()

,Survived
Sex,
female,0.742038
male,0.188908


In [73]:
titanic["RelativesOnboard"] = titanic["SibSp"] + titanic["Parch"]
titanic[["RelativesOnboard", "Survived"]].groupby(
    ['RelativesOnboard']).mean()

,Survived
RelativesOnboard,
0,0.303538
1,0.552795
2,0.578431
3,0.724138
4,0.200000
5,0.136364
6,0.333333
7,0.000000
10,0.000000


In [74]:
titanic[["AgeBucket", "Sex", "Survived", "RelativesOnboard"]].groupby(['Sex', 'AgeBucket', "RelativesOnboard"]).mean()

Survived
Sex    AgeBucket RelativesOnboard          
female 0.0       0                 0.666667
                 1                 0.750000
                 2                 0.800000
                 3                 0.875000
                 4                 0.000000
                 5                 0.000000
                 6                 0.200000
       15.0      0                 0.755102
                 1                 0.774194
                 2                 0.714286
                 3                 0.800000
                 4                 0.500000
                 5                 1.000000
                 6                 1.000000
                 7                 0.000000
       30.0      0                 0.857143
                 1                 0.842105
                 2                 0.800000
                 3                 1.000000
                 5                 0.000000
                 6                 0.500000
                 7                 0.000000
       45.0      0                 0.727273
                 1                 0.833333
                 2                 1.000000
                 3                 1.000000
                 4                 0.500000
                 5                 0.000000
       60.0      0                 1.000000
                 1                 1.000000
male   0.0       0                 0.000000
                 1                 1.000000
                 2                 1.000000
                 3                 1.000000
                 4                 0.000000
                 5                 0.000000
                 6                 0.333333
                 7                 0.000000
       15.0      0                 0.147651
                 1                 0.214286
                 2                 0.133333
                 3                 0.000000
                 4                 0.000000
                 5                 0.000000
       30.0      0                 0.221053
                 1                 0.208333
                 2                 0.071429
                 3                 0.500000
                 5                 0.000000
                 6                 0.000000
       45.0      0                 0.142857
                 1                 0.363636
                 2                 0.250000
       60.0      0                 0.062500
                 1                 0.000000
                 2                 0.333333
                 5                 0.000000
       75.0      0                 1.000000